In [126]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_squared_error

In [127]:
# returns a distance value between two series
# e.g. s1 = sgmd(eigen1) & s2 = relu(eigen1)
# ranging from k0 to k9 for 1 epoch
def get_distance_epoch_value(s1, s2):
    s1 = s1.abs()
    s2 = s2.abs()
    
#     dist = (s1 - s2).abs()
    
    return np.sqrt(mean_squared_error(s1, s2))

In [128]:
# e.g. activator = 'sgmd'
# returns a list of kernel values for an eigen number for the last epoch
def build_series_1epoch(dataset, activator, eig):
    df_cnn_relu0 = pd.read_csv(dataset + "/results/" + activator + "/cnn_K0.csv")
    df_cnn_relu1 = pd.read_csv(dataset + "/results/" + activator + "/cnn_K1.csv")
    df_cnn_relu2 = pd.read_csv(dataset + "/results/" + activator + "/cnn_K2.csv")
    df_cnn_relu3 = pd.read_csv(dataset + "/results/" + activator + "/cnn_K3.csv")
    df_cnn_relu4 = pd.read_csv(dataset + "/results/" + activator + "/cnn_K4.csv")
    df_cnn_relu5 = pd.read_csv(dataset + "/results/" + activator + "/cnn_K5.csv")
    df_cnn_relu6 = pd.read_csv(dataset + "/results/" + activator + "/cnn_K6.csv")
    df_cnn_relu7 = pd.read_csv(dataset + "/results/" + activator + "/cnn_K7.csv")
    df_cnn_relu8 = pd.read_csv(dataset + "/results/" + activator + "/cnn_K8.csv")
    df_cnn_relu9 = pd.read_csv(dataset + "/results/" + activator + "/cnn_K9.csv")

    df_1 =   pd.Series([  df_cnn_relu0.iloc[-1][eig-1], df_cnn_relu1.iloc[-1][eig-1], 
                          df_cnn_relu2.iloc[-1][eig-1], df_cnn_relu3.iloc[-1][eig-1], 
                          df_cnn_relu4.iloc[-1][eig-1], df_cnn_relu5.iloc[-1][eig-1],
                          df_cnn_relu6.iloc[-1][eig-1], df_cnn_relu7.iloc[-1][eig-1], 
                          df_cnn_relu8.iloc[-1][eig-1], df_cnn_relu9.iloc[-1][eig-1]])
    
    return df_1

In [129]:
# e.g. fcn1 = 'sgmd' & fcn2 = 'relu'
# returns a list of distances between 2 activators for all eigen numbers
def get_distance_1eig_epoch_list(dataset, fcn1, fcn2):
    
    eigen_list = []
    for eigen in range(1, 6):
        s1 = build_series_1epoch(dataset, fcn1, eigen)
        s2 = build_series_1epoch(dataset, fcn2, eigen)
        eigen_list.append(get_distance_epoch_value(s1, s2))
        
#     max_norm = max(eigen_list)
#     for i in range(len(eigen_list)):
#         eigen_list[i] = eigen_list[i] / max_norm
        
    return eigen_list
    

In [142]:
# returns err value between the final epochs
def get_err(dataset, fcn1, fcn2):
    e1 = pd.read_csv(dataset + "/results/" + fcn1 + "/cnn_test_loss.csv")
    e2 = pd.read_csv(dataset + "/results/" + fcn2 + "/cnn_test_loss.csv")
    
    err = np.sqrt(mean_squared_error(e1['0'], e2['0']))
    
#     max_norm = max(err)
#     for i in range(len(err)):
#         err[i] = err[i] / max_norm
    
    return err

In [143]:
dataset = 'mnist'
combs = [('sgmd', 'tanh'), ('tanh', 'relu'), ('relu', 'sgmd'),
         ('dropout/0.4/sgmd', 'dropout/0.4/tanh'), 
         ('dropout/0.4/tanh', 'dropout/0.4/relu'), 
         ('dropout/0.4/relu', 'dropout/0.4/sgmd'),
         ('dropout/0.6/sgmd', 'dropout/0.6/tanh'),
         ('dropout/0.6/tanh', 'dropout/0.6/relu'),
         ('dropout/0.6/relu', 'dropout/0.6/sgmd'),
         ('dropout/0.8/sgmd', 'dropout/0.8/tanh'), 
         ('dropout/0.8/tanh', 'dropout/0.8/relu'), 
         ('dropout/0.8/relu', 'dropout/0.8/sgmd')]

main_list = []
        
for comb in combs:
    main_list.append(get_distance_1eig_epoch_list(dataset, comb[0], comb[1]))
    
main_list = main_list

err_list = []

for comb in combs:
    err_list.append(get_err(dataset, comb[0], comb[1]))
    
err_list = err_list

In [144]:
main_T = np.array(main_list).T

In [145]:
main_T

array([[0.13848627, 0.16417577, 0.26378522, 0.26432294, 0.02856797,
        0.26944908, 0.27511463, 0.05898152, 0.28192842, 0.27904722,
        0.07906149, 0.29312595],
       [0.04778592, 0.0721991 , 0.03329693, 0.05375748, 0.03321741,
        0.0306791 , 0.04320363, 0.042317  , 0.02532014, 0.04892302,
        0.05757979, 0.03101222],
       [0.03632907, 0.04363574, 0.03210434, 0.01712307, 0.01922606,
        0.03131598, 0.023487  , 0.02377539, 0.03350494, 0.02536804,
        0.0345962 , 0.04039627],
       [0.09824804, 0.09276244, 0.05384108, 0.06320572, 0.03807531,
        0.04461123, 0.07396622, 0.05884868, 0.06025539, 0.07386583,
        0.08799517, 0.07199933],
       [0.22931695, 0.11676324, 0.18389842, 0.17782869, 0.04473189,
        0.17542402, 0.18738876, 0.09258406, 0.18955209, 0.20735775,
        0.13598938, 0.2045712 ]])

In [146]:
err_list

[0.6430219935770329,
 0.07908536705272963,
 0.7214445602831846,
 0.29157227949904657,
 0.025457244393199387,
 0.31678333189862956,
 0.3833056826697217,
 0.05307682411228387,
 0.4358113726803491,
 0.5087575673321528,
 0.08426122567393608,
 0.5927081875886903]

In [147]:
print(np.corrcoef(main_T[0], err_list)[0][1])
print(np.corrcoef(main_T[1], err_list)[0][1])
print(np.corrcoef(main_T[2], err_list)[0][1])
print(np.corrcoef(main_T[3], err_list)[0][1])
print(np.corrcoef(main_T[4], err_list)[0][1])

0.6847269021371919
-0.39035485018711974
0.25298168567332535
0.12984688443061376
0.8617906924030374
